# Capstone Project Part 3

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


## Import the data from previous stages

In [2]:
neighborhoods = pd.read_csv('~/DataProjects/Coursera_Capstone/output.csv')

In [3]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


## create a map centering coordinates of these postal codes

In [4]:
latitude, longitude = neighborhoods[['Latitude', 'Longitude']].mean().to_list()

In [5]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [6]:
map

### Add Markers to Map

In [7]:
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 

In [8]:
map

Define My credentials

In [9]:
with open('credentials.json', 'r') as credentials:
    creds = credentials.read()
    FOLIUM_CLIENT_ID = json.loads(creds)['FOLIUM_CLIENT_ID']
    FOLIUM_CLIENT_SECRET = json.loads(creds)['FOLIUM_CLIENT_SECRET']

VERSION = '20200601'
LIMIT = 100

### Try to get nearby venues

In [10]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            FOLIUM_CLIENT_ID, 
            FOLIUM_CLIENT_SECRET, 
            VERSION, 
            neighborhoods.loc[2, 'Latitude'],
            neighborhoods.loc[2, 'Longitude'],
            500,
            LIMIT)

In [11]:
venues = requests.get(url).json()

In [12]:
venues["response"]["groups"][0]['items'][0]['venue']

{'id': '55ba864b498ef4d3ce21c633',
 'name': 'Canadian Carpet Cleaning and Janitorial Service',
 'location': {'address': '245 Manse Road',
  'lat': 43.767731354853964,
  'lng': -79.17423963546753,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.767731354853964,
    'lng': -79.17423963546753}],
  'distance': 233,
  'postalCode': 'M1E 4X7',
  'cc': 'CA',
  'city': 'Scarborough',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['245 Manse Road', 'Scarborough ON M1E 4X7', 'Canada']},
 'categories': [{'id': '52f2ab2ebcbc57f1066b8b1d',
   'name': 'Dry Cleaner',
   'pluralName': 'Dry Cleaners',
   'shortName': 'Dry Cleaner',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/drycleaner_',
    'suffix': '.png'},
   'primary': True}],
 'photos': {'count': 0, 'groups': []}}

## get venues nearby for all neighborhoods

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            FOLIUM_CLIENT_ID, 
            FOLIUM_CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [15]:
neighborhoods_venues.shape

(2408, 7)

In [16]:
neighborhoods_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.81153,-79.19552,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Canadian Carpet Cleaning and Janitorial Service,43.767731,-79.174240,Dry Cleaner
3,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.76575,-79.17520,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


In [17]:
pd.set_option('display.max_rows', None)
neighborhoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,8,8,8,8,8,8
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,66,66,66,66,66,66
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


number of unique categories:

In [18]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 263 uniques categories.


### Analyze Each Neighborhood

In [19]:
# one hot encoding
onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
onehot.shape

(2408, 263)

In [21]:
toronto_grouped = onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Berczy Park,0.015152,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.015152,0.000000,...,0.000000,0.000000,0.015152,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.011905,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.011905,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.00000,0.020000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.000000,0.020000,0.000000,0.0000,0.000000,0.010000,0.000000,0.000000,0.000000
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.013889,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.013889,0.000000


print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

       Skating Rink   0.2
1                   Café   0.2
2        College Stadium   0.2
3  General Entertainment   0.2
4                    Gym   0.2


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1          Bar  0.06
2  Coffee Shop  0.06
3   Restaurant  0.05
4    Gift Shop  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.05
2                 Café  0.03
3  Japanese Restaurant  0.02
4        Burrito Place  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2                Café  0.04
3                 Bar  0.04
4                Park  0.03


----Caledonia-Fairbanks----
                 venue  freq
0               Bakery  0.14
1   Mexican Restaurant  0.1

create the new dataframe and display the top 10 venues for each neighborhood.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Park,Pool,Shopping Mall,Supermarket,Sushi Restaurant,Discount Store,Badminton Court,Skating Rink,Costume Shop,Creperie
1,"Alderwood, Long Branch",Sandwich Place,Pizza Place,Gym,Convenience Store,Dance Studio,Pub,Women's Store,Eastern European Restaurant,Dog Run,Donut Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",IT Services,Bar,Park,Furniture / Home Store,Women's Store,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School
3,Bayview Village,Trail,Golf Driving Range,Construction & Landscaping,Park,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Pub,Sushi Restaurant,Sports Club,Juice Bar,Indian Restaurant


## Cluster Neighborhoods

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552,1.0,Home Service,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Event Space
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871,0.0,Bar,Women's Store,Dive Bar,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520,0.0,Construction & Landscaping,Gym / Fitness Center,Park,Dry Cleaner,Women's Store,Elementary School,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
3,M1G,Scarborough,Woburn,43.76820,-79.21761,0.0,Korean Restaurant,Park,Business Service,Indian Restaurant,Coffee Shop,Women's Store,Electronics Store,Donut Shop,Dry Cleaner,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944,3.0,Playground,Women's Store,Elementary School,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


### visualize the result

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if not pd.isnull(cluster):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,Women's Store,Dive Bar,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,Scarborough,0.0,Construction & Landscaping,Gym / Fitness Center,Park,Dry Cleaner,Women's Store,Elementary School,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store
3,Scarborough,0.0,Korean Restaurant,Park,Business Service,Indian Restaurant,Coffee Shop,Women's Store,Electronics Store,Donut Shop,Dry Cleaner,Eastern European Restaurant
5,Scarborough,0.0,Ice Cream Shop,Train Station,Restaurant,Women's Store,Eastern European Restaurant,Distribution Center,Dive Bar,Dog Run,Donut Shop,Dry Cleaner
6,Scarborough,0.0,Coffee Shop,Light Rail Station,Chinese Restaurant,Department Store,Discount Store,Convenience Store,Hobby Shop,Farmers Market,Farm,Falafel Restaurant
7,Scarborough,0.0,Bus Line,Intersection,Bakery,Soccer Field,Bus Station,Coffee Shop,Gym,Metro Station,Falafel Restaurant,Event Space
8,Scarborough,0.0,Ice Cream Shop,Pharmacy,Coffee Shop,Sandwich Place,Liquor Store,Discount Store,Pizza Place,Hardware Store,Event Space,Ethiopian Restaurant
9,Scarborough,0.0,Café,College Stadium,Skating Rink,General Entertainment,Gym,Eastern European Restaurant,Dive Bar,Dog Run,Donut Shop,Dry Cleaner
10,Scarborough,0.0,Bakery,Brewery,Gift Shop,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Women's Store
11,Scarborough,0.0,Auto Garage,Convenience Store,Women's Store,Ethiopian Restaurant,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Event Space


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Home Service,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Event Space
63,Central Toronto,1.0,Home Service,Clothing Store,Women's Store,Elementary School,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2.0,Music Venue,Women's Store,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School


In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3.0,Playground,Women's Store,Elementary School,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,4.0,Movie Theater,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School
